Autor: JESUS ALEJANDRO COLIN VILCHIS

target:
  Analysis of the gisaid data, until July 31.

The necessary libraries are imported

In [1]:
!pip install plotly
!pip install pandas
!pip install numpy
!pip install openpyxl
!pip install --upgrade plotly

import pandas as pd
import plotly.express as px
import re
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Reading the metadata

In [2]:
path='base_nueva_version_11012022.xlsx'
df=pd.read_excel(path)
df=df.iloc[:43183]

Initializing the variables to use

In [3]:
protein=['Spike_','E_','N_','M_','NSP']
clado = {clado:[] for clado in df['Clade'].unique()}
Parts ={
'Spike':[
          ['S1A domain',1,302],
          ['S1A-S1B linker',303,332],
          ['S1B domain',333,527],
          ['S1B - S1C linker',528,533],
          ['S1C domain',534,589],
          ['S1C - S1D linker',590,593],
          ['S1D domain',594,674],
          ['Protease cleavage site',675,692],
          ['S1-S2 subunits linker',693,710],
          ['Central β-strand',711,737],
          ['Downward helix',738,782],
          ['S2’ cleavage site',783,815],
          ['Fusion peptide',816,828],
          ['Connecting region',829,911],
          ['Heptad repeat region 1',912,983],
          ['Central helix',984,1034],
          ['β-hairpin',1035,1068],
          ['β-sheet domain',1069,1133],
          ['Heptad repeat region 2',1134,1213],
          ['Transmembrane region',1214,1236],
          ['Cytoplasmic region',1237,1508]#1273],
], #Guruprasad L. Human SARS CoV-2 spike protein mutations. Proteins. 2021 May;89(5):569-576. doi: 10.1002/prot.26042. Epub 2021 Jan 17. PMID: 33423311; PMCID: PMC8014176
'M':[
      ['N – Terminal (NTD)',1,19],
      ['TMl',20,40],
      ['Mnone',41,50],
      ['TMll',51,71],
      ['Mnone',72,79],
      ['TMlll',80,100],
      ['C-Terminal (CTD)',101,222],
 ],
'N':[
      ['N-terminal (NTD)',1,50],
      ['Dominio de unión al ARN (RBD)',51,174],
      ['Enlazador central desordenado predicho (LINK)',175,246],
      ['Dominio de dimerización',247,365],
      ['C-terminal (CTD)',366,419],
 ],
 'E':[
      ['N-termina',1,16],
      ['Transmembranal (TMD)',17,36],
      ['C-Terminal',37,76]
 ]
 }

the number of total mutations are counted

In [4]:
# give me the aminoacid mutation and the segment where it occurred for each user of the database, for each mutated amino acid after cleaning it, for each protein segment only when the segment is in the mutated aminoacid
amino={'Spike_':[],'E_':[],'N_':[],'M_':[],'NSP':[]}
for user in df['AA Substitutions'].tolist():
  for AA in user.replace(r'(','').replace(r')','').split(','):
    for segment in protein:
      if re.search(segment,AA) is not None:
        amino[segment].append(AA)
        
for segment in amino.keys():
  print(segment,' : ',len(amino[segment]))

for segment in protein: # single mutation count
  value, counts = np.unique(amino[segment], return_counts=True)
  #print(value[1])
  amino[segment]=[(value[i],counts[i],value[i].split('_')[0],value[i].split('_')[1],int(re.findall("\d+",value[i].split('_')[1])[0])) for i in range(len(value))]
  #

Spike_  :  374904
E_  :  1035
N_  :  136997
M_  :  25735
NSP  :  425996


the variety of possible mutations are counted

In [5]:
for segment in amino.keys():
  print(segment,' : ',len(amino[segment]))
  

Spike_  :  2488
E_  :  97
N_  :  913
M_  :  262
NSP  :  10883


The **tables of the regions** are created, but we only show the one for the Spike protein, ***you can visualize some other by changing the 0 to 1,2 or 3***

In [6]:
myDF=[]
for segment in amino.keys():
  myDF.append(pd.DataFrame(amino[segment],columns=[segment,'count','full','change','position'])) 
for i in range(len(myDF)-1):
  myDF[i]['area'] = [x[0] for num in myDF[i].position for x in Parts[myDF[i].full[0]] if num >= x[1] and num <= x[2]]
myDF[0]

,Spike_,count,full,change,position,area
0,Spike_A1016S,7,Spike,A1016S,1016,Central helix
1,Spike_A1016T,1,Spike,A1016T,1016,Central helix
2,Spike_A1016V,1,Spike,A1016V,1016,Central helix
3,Spike_A1020S,70,Spike,A1020S,1020,Central helix
4,Spike_A1020V,32,Spike,A1020V,1020,Central helix
...,...,...,...,...,...,...
2483,Spike_ins256S,1,Spike,ins256S,256,S1A domain
2484,Spike_ins5F,2,Spike,ins5F,5,S1A domain
2485,Spike_ins681KVG,1,Spike,ins681KVG,681,Protease cleavage site
2486,Spike_ins69A,1,Spike,ins69A,69,S1A domain


The NSP table is a special case, but it is elaborated in this section

In [7]:
for i in range(len(myDF)-1):
  myDF[i]['area'] = [x[0] for num in myDF[i].position for x in Parts[myDF[i].full[0]] if num >= x[1] and num <= x[2]]
myDF[4]['area'] = [int(re.findall("\d+",x)[0]) for x in myDF[4].full]
myDF[4].head()

,NSP,count,full,change,position,area
0,NSP10_A104T,99,NSP10,A104T,104,10
1,NSP10_A104V,24,NSP10,A104V,104,10
2,NSP10_A18S,9,NSP10,A18S,18,10
3,NSP10_A18V,1,NSP10,A18V,18,10
4,NSP10_A23S,2,NSP10,A23S,23,10


We filter the 10 most representative views for interaction with the graph.

In [8]:
dic =[mutation for num,segment in enumerate(amino.keys()) for mutation in myDF[num][segment]]

dicS=[mutation for mutation in myDF[0].sort_values('count',ascending=False)['Spike_'][:9]]
dicE=[mutation for mutation in myDF[1].sort_values('count',ascending=False)['E_'][:9]]
dicN=[mutation for mutation in myDF[2].sort_values('count',ascending=False)['N_'][:9]]
dicM=[mutation for mutation in myDF[3].sort_values('count',ascending=False)['M_'][:9]]
dicNSP=[mutation for mutation in myDF[4].sort_values('count',ascending=False)['NSP'][:9]]

From the original table, we retrieve the clades and the week to obtain a table with the clade prevailing over time.

In [16]:


df_clado = df.groupby(['semana','Clade'],as_index=False).size()

existe=list(df_clado.Clade.unique())
for i in range(int(df_clado.semana.max())): # for each week
    for j in range(len(df_clado)): # for the table
        if i == df_clado.iloc[j]['semana']:
            for miclado in df_clado.Clade.unique(): # for the clades
                if miclado == df_clado.iloc[j]['Clade']:
                    clado[miclado].append(df_clado.iloc[j]['size'])
                    existe.remove(df_clado.iloc[j]['Clade'])

    for miclado in existe: # we fill in the empty spaces
        clado[miclado].append(0)
        
    existe=list(df_clado.Clade.unique())
clado=pd.DataFrame(clado)

# Results **display**


In [17]:
fig = px.area(clado,line_shape="spline") # Clade
fig.show()

In [18]:
fig = px.area(clado.divide(clado.sum(axis=1), axis=0),line_shape="spline")
fig.show()

In [19]:
fig =px.sunburst(
    myDF[0],
    path=['area','position','Spike_'],
    values='count',
    title="Spike protein pie graph",
)
fig.show()

In [20]:
fig =px.sunburst(
    myDF[1],
    path=['area','position','E_'],
    values='count',
    title="E protein pie graph",
)
fig.show()

In [22]:
fig =px.sunburst(
    myDF[2],
    path=['area','position','N_'],
    #parents='Tipo de mutacion',
    values='count',
    title="N protein pie graph",
)
fig.show()

In [23]:
fig =px.sunburst(
    myDF[3],
    path=['area','position','M_'],
    values='count',
    title="M protein pie graph",
)
fig.show()

In [28]:
fig =px.sunburst(
    myDF[4],
    path=['area','position','NSP'],
    values='count',
    title="NSP protein pie graph",
)
fig.show()

In [31]:
record=[]
val=0
#An iteration is generated for each day of the week up to the maximum
for week in range(1,int(df.semana.max())+1):
  row=[]
  # Existing mutations are iterated
  for mutation in dicS:
    val=0
    # Columns are filtered for each week
    for user in df[df.semana == week]['AA Substitutions']:
      # we do a count when the mutation appears in the user
      if mutation in user:
        val=val+1
    row.append(val)
  record.append(row)
# to corroborate the results, the protein S dataframe is printed
dfS=pd.DataFrame(record,columns=dicS)
fig = px.line(dfS, 
              x=range(1,int(df.semana.max())+1), 
              y=dfS.columns,line_shape="spline",
              title="Spike protein time-line",)
fig.update_layout(
    xaxis_title="Weeks elapsed",
    yaxis_title="Number of repetitions",
    legend_title="Mutations",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()

In [32]:
record=[]
val=0
for week in range(1,int(df.semana.max())+1):
  row=[]
  for mutation in dicM:
    val=0
    for user in df[df.semana == week]['AA Substitutions']:
      if mutation in user:
        val=val+1
    row.append(val)      
  record.append(row)
dfM=pd.DataFrame(record,columns=dicM)
fig = px.line(dfM, 
              x=range(1,int(df.semana.max())+1), 
              y=dfM.columns,line_shape="spline",
              title="M protein time-line",)
fig.update_layout(
    xaxis_title="Weeks elapsed",
    yaxis_title="Number of repetitions",
    legend_title="Mutations",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()

In [34]:
record=[]
val=0
for week in range(1,int(df.semana.max())+1):
  row=[]
  for mutation in dicN:
    val=0
    for user in df[df.semana == week]['AA Substitutions']:
      if mutation in user:
        val=val+1
    row.append(val)
      
  record.append(row)
  

#Writing the table to the dataframe structure
dfN=pd.DataFrame(record,columns=dicN)

# organizing the chart
fig = px.line(dfN, 
              x=range(1,int(df.semana.max())+1), 
              y=dfN.columns,line_shape="spline",
              title="N protein time-line",)
fig.update_layout(
    xaxis_title="Weeks elapsed",
    yaxis_title="Number of repetitions",
    legend_title="Mutations",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()

In [35]:
record=[]
val=0
for week in range(1,int(df.semana.max())+1):
  row=[]
  for mutation in dicE:
    val=0
    for user in df[df.semana == week]['AA Substitutions']:
      if mutation in user:
        val=val+1
    row.append(val)
  record.append(row)
  

dfE = pd.DataFrame(record,columns=dicE)
fig = px.line(dfE, 
              x=range(1,int(df.semana.max())+1), 
              y=dfE.columns,line_shape="spline",
              title="E protein time-line",)
fig.update_layout(
    xaxis_title="Weeks elapsed",
    yaxis_title="Number of repetitions",
    legend_title="Mutations",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()

In [36]:
record=[]
val=0
for week in range(1,int(df.semana.max())+1):
  row=[]
  for mutation in dicNSP:
    val=0
    for user in df[df.semana == week]['AA Substitutions']:
      if mutation in user:
        val=val+1
    row.append(val)
  record.append(row)
  

dfNSP = pd.DataFrame(record,columns=dicNSP)
fig = px.line(dfNSP, 
              x=range(1,int(df.semana.max())+1), 
              y=dfNSP.columns,line_shape="spline",
              title="NSP protein time-line",)

fig.show()

In [37]:
record=[]
val=0
for week in range(1,int(df.semana.max())+1):
  row=[]
  for mutation in dic:
    val=0
    for user in df[df.semana == week]['AA Substitutions']:
      if mutation in user:
        val=val+1
    row.append(val)
  record.append(row)


In [ ]:
df1=pd.DataFrame(record,columns=dic)
fig = px.line(df1, 
              x=range(1,int(df.semana.max())+1), 
              y=df1.columns,
              title="Full protein time-line",)
fig.show()